In [6]:
import os
import numpy as np
import pandas as pd
import math
import random
from tqdm.auto import tqdm

In [66]:
random.seed(1)
np.random.seed(1)

In [10]:
def generate_pair(csv_file, out_file):
    train_df = pd.read_csv(csv_file)
    font_sample = train_df.font_label.value_counts()
    min_combine = min(font_sample)
    n_combine = math.comb(min_combine, 2)
    print(min_combine, n_combine)
    return
    
    sample_1 = []
    sample_2 = []
    labels = [] # 0: same, 1: diff
    for i in tqdm(font_sample.index):
        fonts = train_df[train_df.font_label == i].path.to_list()

        # choose same
        for x1 in range(min_combine):
            sample_1.extend([fonts[x1]] * len(range(x1 + 1, min_combine)))
            for x2 in range(x1 + 1, min_combine):
                sample_2.append(fonts[-x2])
        labels.extend([0] * n_combine)

        # random diff
        diff_fonts = train_df[train_df != i].path.to_list()
        sample_1.extend(random.choices(fonts, k = n_combine))
        sample_2.extend(random.choices(diff_fonts, k = n_combine))
        labels.extend([1] * n_combine)

    print(len(sample_1), len(sample_2), len(labels))
    df = pd.DataFrame({
        'sample_1': sample_1,
        'sample_2': sample_2,
        'lables': labels
    })

    df.to_csv(out_file, index = False)
    
generate_pair('../data/test_font_all.csv', '../data/test_pair_font_all.csv')

12 66


## stat

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train_pair_font_all.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828764 entries, 0 to 1828763
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   sample_1  object
 1   sample_2  object
 2   lables    int64 
dtypes: int64(1), object(2)
memory usage: 41.9+ MB


In [3]:
df = pd.read_csv('../data/val_pair_font_all.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44604 entries, 0 to 44603
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sample_1  44604 non-null  object
 1   sample_2  44604 non-null  object
 2   lables    44604 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [4]:
df = pd.read_csv('../data/test_pair_font_all.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140184 entries, 0 to 140183
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sample_1  140184 non-null  object
 1   sample_2  140184 non-null  object
 2   lables    140184 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 3.2+ MB
